In [ ]:
# restore dataprep.ipynb variables

%store -r n_user
%store -r n_item

print(n_user)
print(n_item)

In [ ]:
#Tensorflow Install for Notebook
!pip install --upgrade pip && pip install tensorflow==2.2.0

In [ ]:
# import requirements
import os
import json
import sagemaker
import tensorflow as tf
import pandas as pd
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

# get current SageMaker session's execution role and default bucket name
sagemaker_session = sagemaker.Session()

role = get_execution_role()
print("execution role ARN:", role)

bucket_name = sagemaker_session.default_bucket()
print("default bucket name:", bucket_name)

In [ ]:
# specify the location of the training data
training_data_uri = os.path.join(f's3://{bucket_name}', 'data')

In [ ]:
# inspect the training script using `pygmentize` magic
!pygmentize 'ncf.py'

In [ ]:
# specify training instance type and model hyperparameters
# note that for the demo purpose, the number of epoch is set to 1

num_of_instance = 1                 # number of instance to use for training
instance_type = 'ml.c5.2xlarge'     # type of instance to use for training

training_script = 'ncf.py'

training_parameters = {
    'epochs': 1,
    'batch_size': 256, 
    'n_user': n_user, 
    'n_item': n_item
}

# training framework specs
tensorflow_version = '2.1.0'
python_version = 'py3'
distributed_training_spec = {'parameter_server': {'enabled': True}}

In [ ]:
# initiate the training job using Tensorflow estimator
ncf_estimator = TensorFlow(
    entry_point=training_script,
    role=role,
    instance_count=num_of_instance,
    instance_type=instance_type,
    framework_version=tensorflow_version,
    py_version=python_version,
    distribution=distributed_training_spec,
    hyperparameters=training_parameters
)

In [ ]:
# kick off the training job
ncf_estimator.fit(training_data_uri)

In [ ]:
endpoint_name = 'ncf-books-model'

predictor = ncf_estimator.deploy(initial_instance_count=1,   
                                 instance_type='ml.m4.xlarge',  
                                 endpoint_name=endpoint_name
                                 )  


In [ ]:
# To use the endpoint in another notebook, we can initiate a predictor object as follows
from sagemaker.tensorflow import TensorFlowPredictor

predictor = TensorFlowPredictor(endpoint_name)

In [ ]:
# Define a function to read testing data
def _load_testing_data(base_dir):
    """ load testing data """
    df_test = np.load(os.path.join(base_dir, 'test.npy'))
    user_test, item_test, y_test = np.split(np.transpose(df_test).flatten(), 3)
    return user_test, item_test, y_test

In [ ]:
# read testing data from local
user_test, item_test, test_labels = _load_testing_data('./data/s3/')

# one-hot encode the testing data for model input
with tf.compat.v1.Session() as tf_sess:
    test_user_data = tf_sess.run(tf.one_hot(user_test, depth=n_user)).tolist()
    test_item_data = tf_sess.run(tf.one_hot(item_test, depth=n_item)).tolist()
    
# if you're using Tensorflow 2.0 for one hot encoding
# you can convert the tensor to list using:
# tf.one_hot(uuser_test, depth=n_user).numpy().tolist()

In [ ]:
# make batch prediction  
batch_size = 100  
y_pred = []  
for idx in range(0, len(test_user_data), batch_size):  
    # reformat test samples into tensorflow serving acceptable format  
    input_vals = {  
     "instances": [  
         {'input_1': u, 'input_2': i}   
         for (u, i) in zip(test_user_data[idx:idx+batch_size], test_item_data[idx:idx+batch_size])  
    ]}  
   
    # invoke model endpoint to make inference  
    pred = predictor.predict(input_vals)  
      
    # store predictions  
    y_pred.extend([i[0] for i in pred['predictions']])  


In [ ]:
# let's see some prediction examples, assuming the threshold 
# --- prediction probability view ---
print('This is what the prediction output looks like')
print(y_pred[:5], end='\n\n\n')

# --- user item pair prediction view, with threshold of 0.5 applied ---
pred_df = pd.DataFrame([
    user_test,
    item_test,
    (np.array(y_pred) >= 0.5).astype(int)],
).T

pred_df.columns = ['userId', 'ISBN', 'prediction']

print('We can convert the output to user-item pair as shown below')
print(pred_df.head(), end='\n\n\n')

# --- aggregated prediction view, by user ---
print('Lastly, we can roll up the prediction list by user and view it that way')
print(pred_df.query('prediction == 1').groupby('userId').ISBN.apply(list).head().to_frame(), end='\n\n\n')

In [ ]:
#save predictions as a csv
pred_df.to_csv("predictions.csv")